In [1]:
from models.lstmaamautoencoder import LSTMAAMAutoencoder
from utils.loaders import SigGenDataset
from utils.trainers import AAMTrainer
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import make_classification
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob

In [2]:
files = []
for file in glob.glob("SigGen/*.mat"):
    files.append(str(file))
    
files.sort()

In [3]:
model = LSTMAAMAutoencoder(input_dim=1,latent_dim=128, num_layers=4).to("cuda")
dataset = SigGenDataset(files)
trainers = AAMTrainer(model)

In [ ]:
# model = trainers._train(train_loader = dataset, num_epochs = 10, loss_function=torch.nn.MSELoss(), batch_size = 5)
model.load_state_dict(torch.load("all_waveforms_aam_weights.pt"))

Epoch: 0/9
-----------------------------


In [ ]:
torch.save(model.state_dict(), "all_waveforms_aam_weights.pt")

In [ ]:
latent, target = trainers.extract_latent(dataset)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import make_classification
import pandas as pd

# Load or create your dataset
# Replace this with your actual data loading

# Initialize and fit k-Means clustering
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(latent)

# Create a confusion matrix to evaluate clustering performance
# We need to compute a mapping between cluster labels and true labels
# First, create a confusion matrix to find out how clusters are mapped to true labels
conf_matrix = confusion_matrix(target, clusters)

# Find the optimal permutation of cluster labels to match true labels
from scipy.optimize import linear_sum_assignment

# Solve the linear sum assignment problem
row_ind, col_ind = linear_sum_assignment(-conf_matrix)

# Re-map clusters to the true labels
cluster_to_label = dict(zip(col_ind, row_ind))

# Map clusters to labels
y_pred_mapped = np.array([cluster_to_label[cluster] for cluster in clusters])

# Compute accuracy
accuracy = accuracy_score(target, y_pred_mapped)
print(f"Cluster Accuracy: {accuracy:.2f}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Create a DataFrame for the confusion matrix
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index=[f'True_{i}' for i in range(conf_matrix.shape[0])],
                              columns=[f'Cluster_{i}' for i in range(conf_matrix.shape[1])])


# Plot the confusion matrix using a heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title(f'All Waveforms KMeans - Accuracy {accuracy:.2f}')
plt.xlabel('Predicted Cluster')
plt.ylabel('True Label')

file_path = os.path.join('confusion_matrix', "all_confusion_matrix_heatmap_aam.png")

plt.savefig(file_path, dpi=300)

plt.show()